**Environment Setup**

In [1]:
%pip install pfrl@git+https://github.com/voidful/pfrl.git
%pip install textrl==0.2.15
%pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-o42eun3k/pfrl_707eb97a1d2f43b7a133d6947d0c8699
  Running command git clone --filter=blob:none -q https://github.com/voidful/pfrl.git /tmp/pip-install-o42eun3k/pfrl_707eb97a1d2f43b7a133d6947d0c8699
  Resolved https://github.com/voidful/pfrl.git to commit bd679022d91902a3c88066d2646ad69b30d5684a
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans")
model = AutoModelForSeq2SeqLM.from_pretrained("lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans")
model.eval()
model.cuda()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(59481, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(59481, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [3]:
import pfrl
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import torch
from NISQA.nisqa.NISQA_model import nisqaModel

logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

In [4]:
base_path = '/work/b0990106x/TextRL'

# print("model", model)
# print("tokenizer", tokenizer)

**RL Environment: NISQA**

In [5]:
import gym
import logging
import random
import sys
import torch
from torch import autocast
from vc.wav_to_arrow import process_audio

class VcRLEnv(gym.Env):
    def __init__(self, model, tokenizer, observation_input=[], max_length=100, compare_sample=2, unfreeze_layer_from_past=0, env_input_dir=None, env_output_dir=None, instruction="", transcription=""):
        self.model = model
        self.tokenizer = tokenizer
        self.observation_space = observation_input
        self.compare_sample = compare_sample
        self.unfreeze_layer_from_past = 1 if unfreeze_layer_from_past else 0
        self.env_max_length = min(max(self.model.config.max_length, self.tokenizer.model_max_length), max_length)
        self.env_input_dir = env_input_dir
        self.env_output_dir = env_output_dir
        self.instruction = instruction
        self.transcription = transcription
        # self.reset()
        
        self.gen_stop_toks = []
        logging.disable(sys.maxsize)
        if self.tokenizer.sep_token:
            self.gen_stop_toks.append(self.tokenizer.sep_token)
        if self.tokenizer.eos_token:
            self.gen_stop_toks.append(self.tokenizer.eos_token)
        logging.disable(logging.NOTSET)
        
    def step(self, count):
        reward = self._predict(count)
        self._get_obs(count)
        return reward
    
    def get_reward(self, input_path=None, output_dir=None, count=0): # predicted will be the list of predicted token
        args = {
            'mode': 'predict_file', 
            'pretrained_model': f'{base_path}/NISQA/weights/nisqa.tar', 
            'deg': f'{base_path}/NISQA/wav/{count}.wav', 
            'data_dir': None, 
            'output_dir': f'{base_path}/NISQA/result',
            'csv_file': None, 
            'csv_deg': None,  
            'num_workers': 0, 
            'bs': 1,
            'ms_channel': None
        }

        if input_path is not None:
            args['deg'] = input_path

        args['tr_bs_val'] = args['bs']
        args['tr_num_workers'] = args['num_workers']
        
        nisqa = nisqaModel(args)
        prediction = nisqa.predict()
        reward = float(prediction['mos_pred'].iloc[0])
        return reward
    
    # @autocast('cuda')
    # def reset(self, input_item=None):
    #     self.predicted = [[]] * self.compare_sample
    #     self.predicted_end = [False] * self.compare_sample
    #     self.input_item = {"input": ""}
    #     return self._get_obs(self.predicted)
    
    def _get_obs(self, count):
        # get the observation
        audio_path = f"{self.env_input_dir}/{count}.wav"

        process_audio(source_audio_path=audio_path, output_dir = self.env_output_dir, temp_dir="/work/b0990106x/TextRL/vc/data/temp", instruction=self.instruction, transcription=self.transcription)
        return count
        
    def _predict(self, count):
        self.get_reward(count)
        
        
        


PyTorch version 2.2.1 available.


In [52]:
from torch import autocast
from transformers import (AutoTokenizer, BartForConditionalGeneration,
                          BatchEncoding)
from vc.trainer_encodec_vc_inference import cascade_ar_nar, convert_to_encode_code,synthesize_audio
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
import numpy as np
import soundfile as sf
import textrl.actor

import itertools

import numpy as np
import pfrl
import torch
import torch.nn.functional as F
from pfrl.agents.ppo import _elementwise_clip
from pfrl.utils.mode_of_distribution import mode_of_distribution
from torch import autocast


class VcPPOAgent(pfrl.agents.PPO):
    def __init__(self, model, tokenizer, device="cuda", observation_input=[], max_length=100, compare_sample=2):
        env = VcRLEnv(model, tokenizer, device, observation_input, max_length, compare_sample)
        model = VcActor(model, tokenizer, device, observation_input, max_length, compare_sample)
        super().__init__(env=env, model=model)
        
class VcActor(TextRLActor):
    def __init__(self, model, tokenizer,ar_checkpoint, nar_checkpoint, input_dir, output_dir, device, observation_input=[], max_length=100, compare_sample=2):
        
        
        self.model = model
        self.tokenizer = tokenizer
        self.observation_input = observation_input
        self.ar_tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
        self.ar_model = BartForConditionalGeneration.from_pretrained(ar_checkpoint)
        self.nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
        self.nar_model = NARBartForConditionalGeneration.from_pretrained(nar_checkpoint)
        self.device = torch.device(device)
        self.ar_model.to(self.device)
        self.input_dir = input_dir
        self.output_dir = output_dir

    @autocast('cuda')
    def predict(self, count):
        # use the model to predict the next token
        layer_list = cascade_ar_nar(self.ar_model, self.nar_model, self.ar_tokenizer, self.nar_tokenizer, self.input_dir, self.device)
        encodec_code = convert_to_encode_code(self.nar_tokenizer, layer_list)    
        audio = synthesize_audio(encodec_code, self.device)
        output_path = f"{self.output_dir}/{count}.wav"
        sf.write(output_path, np.ravel(audio), samplerate=24000)

    def agent_ppo(self, update_interval=10, minibatch_size=3000, epochs=20, lr=3e-6):
        policy = torch.nn.Sequential(
            self.middle_model,
            self.remaining_model,
            self.converter,
            textrl.actor.SoftmaxCategoricalHead(self.env,
                                   temperature=self.temperature,
                                   top_k=self.top_k,
                                   top_p=self.top_p)
        )
        vf = torch.nn.Sequential(
            torch.nn.Linear(self.obs_size, self.obs_size // 2),
            torch.nn.Linear(self.obs_size // 2, self.obs_size // 4),
            torch.nn.Linear(self.obs_size // 4, 1)
        )
        model = pfrl.nn.Branched(policy, vf)
        if isinstance(self.optimizer, str):
            if self.optimizer.lower() == 'adamw':
                opt = torch.optim.AdamW(model.parameters(), lr=lr)
            else:
                opt = torch.optim.SGD(model.parameters(), lr=lr)
        else:
            opt = self.optimizer
        model = model.cuda()
        agent = textrl.actor.TextPPO(
            model,
            opt,
            gpu=self.gpu_id,
            update_interval=update_interval,
            minibatch_size=minibatch_size,
            epochs=epochs,
            clip_eps_vf=None,
            entropy_coef=0,
            gamma=0.95,  # https://arxiv.org/abs/2210.01241
            lambd=1,
            max_grad_norm=1.0,
            standardize_advantages=True,
            act_deterministically=self.act_deterministically
        )
        self.agent = agent
        return agent


    

**RL Agent: Text-Instruction-Guided Voice Conversion Model**

In [53]:
import random
import numpy as np
import soundfile as sf
import torch
from datasets import load_dataset, load_from_disk
from encodec import EncodecModel
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
from argparse import ArgumentParser, Namespace
from transformers import (AutoTokenizer, BartForConditionalGeneration,
                          BatchEncoding)
import vc.trainer_encodec_vc_inference as vc_inference
from types import SimpleNamespace

args = SimpleNamespace(
    dataset="lca0503/soxdata_small_encodec",
    splits=["train"],
    ground_truth_only=False,
    cascade_ar_nar=True,
    nar_model_only=False,
    ground_truth_model_name="voidful/bart-base-unit",
    ar_checkpoint="lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans",
    nar_checkpoint="lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans",
    ground_truth_output_path="output_wav/vc/ground_truth/train_1.wav",
    cascade_output_path="output_wav/vc/ar_nar_cascade/train_1.wav",
    nar_output_path="output_wav/vc/nar/train_1.wav",
    seed=0,
    device="cuda"
)

# modify ar_checkpoint in args
args.ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"




**Load Datasets**

In [54]:
# %pip install datasets
# from datasets import load_from_disk ,load_dataset

# dataset = load_dataset("lca0503/soxdata_encodec")
# dataset.save_to_disk("data")

# dataset = load_dataset("lca0503/soxdata_encodec", split="+".join(["train"]))
# dataset = dataset.filter(lambda x : len(x[f"src_encodec_0"]) <= 700)
# dataset = dataset.shuffle(0).select(range(1))

# dataset.save_to_disk("data-encodec")
# dataset = load_from_disk("data-encodec")

**Start Training**

*1. Agent to Environment*

In [55]:
# define path
agent_input_dir = f'{base_path}/data-encodec'
agent_output_dir = f'{base_path}/output'
env_input_dir = agent_output_dir
env_output_dir = agent_input_dir

ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

In [56]:
device = "cuda" if torch.cuda.is_available() else "cpu"
ar_tokenizer = AutoTokenizer.from_pretrained(args.ar_checkpoint)
ar_model = BartForConditionalGeneration.from_pretrained(args.ar_checkpoint)
ar_model.to(device)

dataset = load_from_disk(agent_input_dir)
instruction_ids = ar_tokenizer(dataset["instruction"][0])["input_ids"][1 : -1]
transcription_ids = ar_tokenizer(dataset["transcription"][0])["input_ids"][1 : -1]
instruction = dataset["instruction"][0]
transcription = dataset["transcription"][0]

for i in range(len(instruction_ids)):
    print("Instruction(cascade): ", ar_tokenizer.decode(instruction_ids[i]))
for i in range(len(transcription_ids)):
    print("Transcription(cascade): ", ar_tokenizer.decode(transcription_ids[i]))
    
observation_list = [{'input': 0, 'transcription': transcription, 'instruction': instruction, 'dataset': dataset}]

In [57]:

actor = VcActor(model, tokenizer, ar_checkpoint, nar_checkpoint, dataset, agent_output_dir, device, observation_list, 100, 2)

actor.predict(0)

AttributeError: 'VcActor' object has no attribute 'device'

In [ ]:
# define env, actor, and agent
env = VcRLEnv(model, tokenizer ,observation_input=observation_list, max_length=100, compare_sample=2, transcription=transcription, instruction=instruction)
# actor = TextRLActor(env,model,tokenizer)

actor = VcActor(model, tokenizer, ar_checkpoint, nar_checkpoint, dataset, agent_output_dir, device, observation_list, 100, 2)


for i in range(10):
    print("Step: ", i)
    actor.predict(i)
    env.step(i)
    actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)
    
# agent = vc_inference.run(args, agent_input_dir, agent_output_path)

Step:  0
Dataset Dataset({
    features: ['file_id', 'instruction', 'transcription', 'src_encodec_0', 'src_encodec_1', 'src_encodec_2', 'src_encodec_3', 'src_encodec_4', 'src_encodec_5', 'src_encodec_6', 'src_encodec_7', 'tgt_encodec_0', 'tgt_encodec_1', 'tgt_encodec_2', 'tgt_encodec_3', 'tgt_encodec_4', 'tgt_encodec_5', 'tgt_encodec_6', 'tgt_encodec_7'],
    num_rows: 4980
})
[47302, 5, 723, 32407, 11, 10, 20228, 4737, 4]
[113, 534, 5229, 11, 289, 13255, 523, 2901, 37, 15355, 4]
Instruction:  Improve
Instruction:   the
Instruction:   higher
Instruction:   frequencies
Instruction:   in
Instruction:   a
Instruction:   noticeable
Instruction:   manner
Instruction:  .
Transcription:  "
Transcription:  G
Transcription:  ott
Transcription:   in
Transcription:   H
Transcription:  imm
Transcription:  el
Transcription:  !"
Transcription:   he
Transcription:   shouted
Transcription:  .


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

NameError: name 'pfrl' is not defined

In [ ]:
# actor.predict(observaton_list[0])

*2. Environment to Agent*

In [ ]:
# reward = env.get_reward()
# print("reward", reward)
env._predict()

Device: cuda
Model architecture: NISQA_DIM
Loaded pretrained model from /work/b0990106x/TextRL/NISQA/weights/nisqa.tar
reward 3


/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


In [ ]:
# output of agent (wav) + instruction + transcription